In [297]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F

from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [298]:
df = pd.read_csv("Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [299]:
df = df.drop(["Id"], axis=1)
# df.columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [300]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [301]:
# ohe_df = pd.get_dummies(data=df, columns=list(df.select_dtypes(['object']).columns))
df['Species'] = df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica':2})
df.head()
# ohe_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [302]:
# X = ohe_df.iloc[:100,[0,1,2,3]].values
# y = ohe_df.iloc[:100,[4, 5, 6]].values

# X = df.iloc[:150,[0,1,2,3]].values
# y = df.iloc[:150,[4]].values

X = df.drop('Species', axis=1).values
y = df['Species'].values

In [303]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
X_scaled = X

In [304]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 0)

In [305]:
y_train

array([2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 2, 2, 1, 2,
       1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0,
       1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1,
       0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1,
       1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

In [306]:
# class Perceptron(nn.Module):
#     def __init__(self):
#         super(Perceptron, self).__init__()
#         self.fc = nn.Linear(1,1)
#         self.relu = nn.ReLU() # instead of Heaviside step fn
#     def forward(self, x):
#         output = self.fc(x)
#         output = self.relu(x) # instead of Heaviside step fn
#         return output

In [307]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [308]:
class Model(nn.Module):
    def __init__(self, input_features=4, hidden_layer1=25, hidden_layer2=30, output_features=3):
        super().__init__()
        self.fc1 = nn.Linear(input_features,hidden_layer1)                  
        self.fc2 = nn.Linear(hidden_layer1, hidden_layer2)                  
        self.out = nn.Linear(hidden_layer2, output_features)      
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [309]:
model     = Model(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer.zero_grad()
model

Model(
  (fc1): Linear(in_features=4, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=30, bias=True)
  (out): Linear(in_features=30, out_features=3, bias=True)
)

In [310]:
y_train

tensor([2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1,
        2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1,
        0, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0,
        1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0,
        2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

In [311]:
epochs = 100
losses = []

for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  0  loss: 1.18489444
epoch:  1  loss: 1.17247140
epoch:  2  loss: 1.15189552
epoch:  3  loss: 1.12865186
epoch:  4  loss: 1.10690367
epoch:  5  loss: 1.08900225
epoch:  6  loss: 1.07601821
epoch:  7  loss: 1.06713784
epoch:  8  loss: 1.06093824
epoch:  9  loss: 1.05605543
epoch: 10  loss: 1.05102456
epoch: 11  loss: 1.04493308
epoch: 12  loss: 1.03696823
epoch: 13  loss: 1.02698410
epoch: 14  loss: 1.01528680
epoch: 15  loss: 1.00246489
epoch: 16  loss: 0.98906517
epoch: 17  loss: 0.97548616
epoch: 18  loss: 0.96184653
epoch: 19  loss: 0.94793797
epoch: 20  loss: 0.93326932
epoch: 21  loss: 0.91719377
epoch: 22  loss: 0.89917952
epoch: 23  loss: 0.87905866
epoch: 24  loss: 0.85717142
epoch: 25  loss: 0.83420628
epoch: 26  loss: 0.81075072
epoch: 27  loss: 0.78763950
epoch: 28  loss: 0.76519233
epoch: 29  loss: 0.74234116
epoch: 30  loss: 0.71753448
epoch: 31  loss: 0.69087154
epoch: 32  loss: 0.66399521
epoch: 33  loss: 0.63872129
epoch: 34  loss: 0.61570591
epoch: 35  loss: 0.5

In [312]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [313]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]

In [314]:
df['Correct'].sum() / len(df)

1.0